In [1]:
import paramiko
import getpass

host = "158.251.88.197"
port = 22
username = "iaraya"
password = getpass.getpass()

ssh = paramiko.SSHClient()
ssh.set_missing_host_key_policy(paramiko.AutoAddPolicy())
ssh.connect(host, port, username, password)

 ········


### Loading the instance

In [3]:
from bsg import load_LargeInstance
from mclp import generate_candidate_solution
import numpy as np

#L,W,H,boxes,id2box = load_BRinstance(filename="BR8.txt",inst=4, nbox=1)
#L,W,H,boxes,id2box = load_BRKGAinstance(filename="../benchs/class7/200.txt",inst=1, nbox=1)
L,W,H,boxes,id2box = load_LargeInstance(filename="../benchs/Data_large/L_500/L_500_2.txt", nbox=1, rot_allowed=True)
_boxes=boxes.copy()

#Dimension del contenedor
#L = int(L/2)
#W = int(W/2)
#H = int(H/2)


r_param = 1.5 
total_test = 100

### Estimating adjusted vols (optional)

In [4]:
from sklearn.linear_model import LinearRegression

boxes=_boxes.copy()
#Creacion de solucion candidata
bins1 = generate_candidate_solution(ssh,L,W,H,boxes,id2box, r_param=r_param, bsg_time=1)
boxes=_boxes.copy()
bins2 = generate_candidate_solution(ssh,L,W,H,boxes,id2box, r_param=r_param, bsg_time=1)
boxes=_boxes.copy()
bins3 = generate_candidate_solution(ssh,L,W,H,boxes,id2box, r_param=r_param, bsg_time=1)
boxes=_boxes.copy()
bins4 = generate_candidate_solution(ssh,L,W,H,boxes,id2box, r_param=r_param, bsg_time=1)
boxes=_boxes.copy()
bins5 = generate_candidate_solution(ssh,L,W,H,boxes,id2box, r_param=r_param, bsg_time=1)

# obtengo conjuntos de bins que se pueden colocar en un bin
bins=[*bins1[:-2],*bins2[:-2],*bins3[:-2],*bins4[:-2],*bins5[:-2]] #40 bins

#10x1 + 5x2 +7x3 = 1
#7x3 + 1x7 = 1
#7x3 + 14x1 = 1 
#...


X = []; y=[]
for bin in bins:
    x = np.zeros(len(_boxes))
    i=0
    for box in _boxes:
        if box in bin.boxes:
            x[i]=bin.boxes[box]
        i+=1
    X.append(x)
    y.append(1.0)

reg = LinearRegression(fit_intercept=False).fit(X, y)

i = 0

for box in _boxes:
    vol = box.vol/(L*W*H) 
    if reg.coef_[i] < vol: box.adj_vol=vol
    else: box.adj_vol=reg.coef_[i]
    i+=1
    
print( reg.coef_)

Initial Solution: 4
Initial Solution: 4
Initial Solution: 4
Initial Solution: 4
Initial Solution: 4
[5.10799304e-03 1.28767779e-02 7.76688132e-03 7.00650849e-03
 1.93890361e-03 7.47080958e-03 7.89171299e-03 6.93889390e-17
 2.42861287e-17 9.88173869e-03 7.55556203e-03 1.20399538e-02
 5.40031650e-03 7.14031047e-03 7.50432232e-03 4.47577035e-03
 9.40093420e-03 7.89171299e-03 1.01345629e-02 2.99425245e-03
 0.00000000e+00 1.24464264e-02 9.70578493e-03 0.00000000e+00
 1.00046668e-02 9.92212685e-03 2.12094601e-03 2.99425245e-03
 1.22562957e-02 4.05984962e-03 9.36133283e-03 9.59662065e-03
 2.11295381e-03 6.93507981e-03 1.23563721e-02 7.27044399e-03
 5.14949798e-03 1.01002634e-02 8.83169170e-03 5.65314072e-03
 2.12094601e-03 6.60236820e-03 1.00015477e-02 6.83624653e-03
 9.40971319e-03 1.02042456e-02 9.91822099e-03 0.00000000e+00
 1.20399538e-02 7.26323857e-03 5.14949798e-03 0.00000000e+00
 4.68100100e-03 4.77184208e-03 7.77408673e-03 7.89092620e-03
 0.00000000e+00 7.50432232e-03 6.80194701e-03 

In [5]:
#precomputed_vols = [ 0.11630617,  0.02313203, -0.00057166,  0.02554413,  0.03971392,  0.05794329, \
#  0.03677449,  0.04732551,  0.11323912,  0.08207787,  0.02197904,  0.17862389, \
#  0.05774166,  0.14886613, -0.03230753,  0.03970321,  0.09253112,  0.0095023, \
#  0.10369953,  0.04723096,  0.09009306,  0.06664873,  0.07504355,  0.05946155, \
#  0.04751135,  0.08771801,  0.05591064,  0.22794829,  0.07499608,  0.03502724]
i=0


precomputed_vols = reg.coef_
for box in _boxes:
    vol = box.vol/(L*W*H) 
    if precomputed_vols[i] < vol: box.adj_vol=vol
    else: box.adj_vol=precomputed_vols[i]
    i+=1

### Solver

In [6]:
from base.Heuristics.mclp import get_random_bin

def get_adjusted_vol(boxes):
    vol=0.0
    for box in boxes:
        vol += box.adj_vol*boxes[box]
    return vol

def get_vol(boxes):
    vol=0.0
    for box in boxes:
        vol += box.vol*boxes[box]
    return vol
      
def adjusted_swap(solution, n=2,  max_vol_accept=1.0, tolerance=0.1, verbose=True):
    bin_A = get_random_bin(s = solution)
    bin_B = get_random_bin(s = solution, b = bin_A)
    
    var_ini = (bin_A.adj_vol-1.0)**2 + (bin_B.adj_vol-1.0)**2
    
    nA = int(random.randint(1, n))
    boxes_A = bin_A.pop_random_boxes(nA)
    adjvolA = get_adjusted_vol(boxes_A)
    
    nB = int(random.randint(0, n))
    boxes_B = bin_B.pop_random_boxes(nB)
    adjvolB = get_adjusted_vol(boxes_B)
    
    r = tolerance*random.random()
    accepted = False
    
    if adjvolA > adjvolB and bin_B.adj_vol + adjvolA < max_vol_accept + r:
        accepted = True
    elif adjvolB > adjvolA and  bin_A.adj_vol + adjvolB < max_vol_accept + r:
        accepted = True
    
    if accepted:
        bin_A.insert_boxes(boxes_B)
        bin_B.insert_boxes(boxes_A)
    else: return -10
        
    var_final = (bin_A.adj_vol-1.0)**2 + (bin_B.adj_vol-1.0)**2
        
    var_diff = var_final - var_ini
    
    return var_diff

    

In [7]:
boxes=_boxes.copy()
init_solution=generate_candidate_solution(ssh,L,W,H,boxes,id2box, r_param=r_param)


Initial Solution: 4


In [8]:
import copy as cp
for bin in init_solution:
    bin.set_adj_vol(get_adjusted_vol(bin.boxes))
    bin.set_vol(get_vol(bin.boxes))
# Bins adjusted volumes
print("adjusted volumes of each bin:",[get_adjusted_vol(bin.boxes) for bin in init_solution])
print([get_vol(bin.boxes) for bin in init_solution])
best_solution = cp.deepcopy(init_solution)

adjusted volumes of each bin: [1.0412740281255797, 1.0209617314280897, 1.0244251316549697, 0.6890749372052494]
[33343971.0, 33018131.0, 32620385.0, 22408135.0]


In [ ]:
from base.Heuristics.mclp import verify_solution

for i in range(1000):
    solution = cp.deepcopy(best_solution)
    diff_var = adjusted_swap(solution, n=2,  max_vol_accept=1.03, tolerance=0.05, verbose=True)

    if diff_var>0:
        print(diff_var)
        verified_solution=True
        #max_bl: maxima cantidad de bloques a generar por bsg
        #min_fr: minimo porcentaje ocupado para considerar la generación de un bloque
        if not verify_solution(ssh, solution, id2box, bsg_time=5, args="--max_bl=10000 --min_fr=0.99"):
            print(i,"verification fails", [(b.adj_vol,b.vol) for b in solution if b.verify==False])
            verified_solution=False

        if verified_solution:
            for bin in solution:
                print(bin.id, ":", bin, bin.utilization, bin.adj_vol, bin.verify)
            best_solution = cp.deepcopy(solution)

0.00817745709681339
1 : [212, 1], [40, 1], [120, 1], [367, 1], [370, 1], [102, 1], [196, 1], [97, 1], [327, 1], [187, 1], [431, 1], [58, 1], [326, 1], [492, 1], [98, 1], [318, 1], [328, 1], [464, 1], [185, 1], [66, 1], [278, 1], [274, 1], [69, 1], [35, 1], [355, 1], [402, 1], [392, 1], [347, 1], [115, 1], [459, 1], [3, 1], [414, 1], [461, 1], [496, 1], [15, 1], [72, 1], [491, 1], [50, 1], [394, 1], [341, 1], [190, 1], [391, 1], [365, 1], [31, 1], [116, 1], [142, 1], [306, 1], [346, 1], [154, 1], [68, 1], [429, 1], [422, 1], [457, 1], [4, 1], [333, 1], [331, 1], [285, 1], [291, 1], [353, 1], [143, 1], [2, 1], [354, 1], [399, 1], [295, 1], [371, 1], [110, 1], [276, 1], [32, 1], [500, 1], [364, 1], [468, 1], [258, 1], [95, 1], [198, 1], [302, 1], [447, 1], [23, 1], [76, 1], [477, 1], [344, 1], [100, 1], [213, 1], [65, 1], [489, 1], [439, 1], [335, 1], [469, 1], [377, 1], [174, 1], [128, 1], [42, 1], [175, 1], [411, 1], [132, 1], [463, 1], [454, 1], [138, 1], [17, 1], [303, 1], [109, 1], [